In [5]:
import requests as re
from bs4 import BeautifulSoup
from os import path
import numpy as np
import pandas as pd

PATH_DIR = r'C:\Users\Никита\Андан\Project\data\macro'

## Росстат

In [2]:
def get_link():

    responce = BeautifulSoup(re.get('https://rosstat.gov.ru/statistics/accounts').content, 'html.parser')
    link = responce.find_all('a', {'class': 'btn btn-icon btn-white btn-br btn-sm'})[1].get('href')

    responce = re.get(r'https://rosstat.gov.ru' + link)
    with open(path.join(PATH_DIR, 'GDP.xlsx'), 'wb') as file:
        file.write(responce.content)
    

In [3]:
get_link()

In [6]:
df = pd.read_excel(path.join(PATH_DIR, 'GDP.xlsx'), 9)
df

К содержанию    Unnamed: 1  \
0  Валовой внутрений продукт 1)\n(в ценах 2021г.,...           NaN   
1                                               2011           NaN   
2                                          I квартал    II квартал   
3                                       26470.295854  28624.554577   
4                                                NaN           NaN   
5  1) Данные содержат изменения, связанные с внед...           NaN   
6  2) Без учета статистической информации по Доне...           NaN   

     Unnamed: 2    Unnamed: 3    Unnamed: 4    Unnamed: 5    Unnamed: 6  \
0           NaN           NaN           NaN           NaN           NaN   
1           NaN           NaN          2012           NaN           NaN   
2   III квартал    IV квартал     I квартал    II квартал   III квартал   
3  30177.235769  32311.819119  27979.628161  30013.804133  31230.625524   
4           NaN           NaN           NaN           NaN           NaN   
5           NaN           NaN           NaN           NaN           NaN   
6           NaN           NaN           NaN           NaN           NaN   

     Unnamed: 7    Unnamed: 8    Unnamed: 9  ...   Unnamed: 42   Unnamed: 43  \
0           NaN           NaN           NaN  ...           NaN           NaN   
1           NaN          2013           NaN  ...           NaN           NaN   
2    IV квартал     I квартал    II квартал  ...   III квартал    IV квартал   
3  33091.525158  28296.087789  30530.548807  ...  34818.893245  38142.716608   
4           NaN           NaN           NaN  ...           NaN           NaN   
5           NaN           NaN           NaN  ...           NaN           NaN   
6           NaN           NaN           NaN  ...           NaN           NaN   

   Unnamed: 44   Unnamed: 45   Unnamed: 46   Unnamed: 47   Unnamed: 48  \
0          NaN           NaN           NaN           NaN           NaN   
1       20222)           NaN           NaN           NaN        20232)   
2    I квартал    II квартал   III квартал    IV квартал     I квартал   
3  31103.95782  31679.223218  33857.522587  37439.930767  30615.796202   
4          NaN           NaN           NaN           NaN           NaN   
5          NaN           NaN           NaN           NaN           NaN   
6          NaN           NaN           NaN           NaN           NaN   

   Unnamed: 49   Unnamed: 50   Unnamed: 51  
0          NaN           NaN           NaN  
1          NaN           NaN           NaN  
2   II квартал   III квартал    IV квартал  
3  33291.76746  35797.406698  39268.666618  
4          NaN           NaN           NaN  
5          NaN           NaN           NaN  
6          NaN           NaN           NaN  

[7 rows x 52 columns]

In [23]:
def process_gdp(path):
    gdp_raw = pd.read_excel(path, 9)
    
    df = pd.DataFrame({})
    df['gdp in 2021 prices'] = gdp_raw.iloc[3,:].astype('float32').to_numpy()
    
    gdp_raw = pd.read_excel(path, 10)
    df['gdp without seasonality in 2021 prices'] = gdp_raw.iloc[3,:].astype('float32').to_numpy()
    
    
    df['year'] = 2011 + np.arange(df.shape[0]) // 4
    df['quater'] = 1+np.arange(df.shape[0]) % 4

    gdp_raw = pd.read_excel(path, 10)
    df['gdp without seasonality in 2021 prices'] = gdp_raw.iloc[3,:].astype('float32').to_numpy()

    return df

In [24]:
df = process_gdp(path.join(PATH_DIR, 'GDP.xlsx'))
df.to_csv(path.join(PATH_DIR, 'GDP_processed.csv'), \
          index = False, encoding = 'utf-8')
df.isna().sum()

gdp in 2021 prices                        0
gdp without seasonality in 2021 prices    0
year                                      0
quater                                    0
dtype: int64

## Парсинг ЦБ

In [9]:
LINKS = ['https://www.cbr.ru/vfs/statistics/households/households_bm.xlsx', \
         'https://www.cbr.ru/vfs/statistics/nfs_hd/nonfinancial_sector_households_debt_extended.xlsx',\
        'https://www.cbr.ru/Content/Document/File/128613/monetary_agg.xlsx']

for link in LINKS:
    res = re.get(link)
    with open(path.join(PATH_DIR, link.split('/')[-1]), 'wb') as file:
        if res.status_code == 200:
            print('OK')
            file.write(res.content)       

OK
OK
OK


In [10]:
def preprocess_balance(path):
    raw_df = pd.read_excel(path, 'Балансы')
    df = pd.DataFrame({})

    columns = ['Наличная национальная валюта', 'Наличная иностранная валюта', 'Депозиты', \
               'Денежные средства на брокерских счетах -всего(5)', 'Долговые ценные бумаги', \
               'Котируемые акции и паи и акции инвестиционных фондов*', 
              'Кредиты кредитных организаций - всего']
    dates = pd.to_datetime(raw_df.loc[2,:].drop('Unnamed: 0'))
    df['date'] = dates
    
    for column in columns:
        df[column] = raw_df[raw_df['Unnamed: 0'] == column].drop('Unnamed: 0', axis = 1).astype('float32').transpose()
    
    df = df.reset_index().drop('index', axis = 1)

    df.rename({'Денежные средства на брокерских счетах -всего(5)': 'Денежные средства на брокерских счетах', 
              'Котируемые акции и паи и акции инвестиционных фондов*': 'Котируемые акции и паи и акции инвестиционных фондов', 
              'Кредиты кредитных организаций - всего': 'Кредиты кредитных организаций'}, axis = 1, inplace = True)
    
    return df

In [25]:
df = preprocess_balance(path.join(PATH_DIR, 'households_bm.xlsx'))
df.to_csv(path.join(PATH_DIR, 'households_bm_processed.csv'), \
         index = False, encoding = 'utf-8')

df.isna().sum()

date                                                    0
Наличная национальная валюта                            0
Наличная иностранная валюта                             0
Депозиты                                                0
Денежные средства на брокерских счетах                  0
Долговые ценные бумаги                                  0
Котируемые акции и паи и акции инвестиционных фондов    0
Кредиты кредитных организаций                           0
dtype: int64

In [12]:
def process_moneraty_mass(path):
    raw_df = pd.read_excel(path, 'Денежные агрегаты')
    df = pd.DataFrame({})
    
    columns = ['Денежный агрегат М0', 'Денежный агрегат М1', 'Денежный агрегат М2', 'Широкая денежная масса']
    for column in columns:
        df[column] = raw_df[raw_df['Денежные агрегаты*, млрд руб.'] == column].iloc[:,97:].transpose().astype('float32')

    df = df.reset_index().rename({'index':'date'}, axis = 1)
    df['date'] = pd.to_datetime(df['date'])

    return df

In [26]:
df = process_moneraty_mass(path.join(PATH_DIR, 'monetary_agg.xlsx'))
df.to_csv(path.join(PATH_DIR, 'monetary_agg_processed.csv'),\
         index = False, encoding = 'utf-8')

df.isna().sum()

date                      0
Денежный агрегат М0       0
Денежный агрегат М1       0
Денежный агрегат М2       0
Широкая денежная масса    0
dtype: int64

In [15]:
def process_debt(path):
    raw_df = pd.read_excel(path)
    column_names = raw_df.iloc[4:15, 1:].transpose().iloc[0,:]
    
    df = raw_df.iloc[4:15, 1:].transpose()\
        .set_axis(column_names, axis = 1) \
        .drop('Unnamed: 1').astype('int32')
    df['date'] = pd.to_datetime(raw_df.iloc[3,2:].transpose())
    df = df.reset_index().drop('index', axis = 1)

    return df


In [19]:
df = process_debt(path.join(PATH_DIR, 'nonfinancial_sector_households_debt_extended.xlsx'))
df.to_csv(path.join(PATH_DIR, 'nonfinancial_sector_households_debt_extended_processed.csv'), \
          index = False, encoding = 'utf-8')
df.isna().sum()

Unnamed: 1
 Долг нефинансового сектора и домашних хозяйств, итого                           0
    Нефинансовый сектор                                                          0
          Внутренние заимствования                                               0
              Кредиты                                                            0
              Долговые ценные бумаги в портфеле резидентов                       0
          Внешние заимствования                                                  0
              Кредиты и займы, полученные от нерезидентов                        0
              Долговые ценные бумаги в портфеле нерезидентов, включая векселя    0
    Домашние хозяйства и НКООДХ                                                  0
          Внутренние заимствования                                               0
          Внешние заимствования                                                  0
date                                                                        